In [1]:
from matplotlib import pyplot as plt
import time
import mouseQPCRModelSelection
import pods
import numpy as np

''' File to run model selection on mouse QPCR data
   Need to set 
   subsetSelection = integer - how many point to skip before next point?
   fPseudoTime     = Boolean - use pseudotime or capture time?
   strgene             = string - which gene to look at
'''
subsetSelection = 3
strgene = 'Id2'
fPseudoTime = True # if false use capture time

print 'Doing subsets selection %g, looking at gene %s and pseudotime=%g'%(subsetSelection,strgene,fPseudoTime)
print 'Loading QPCR data'
data = pods.datasets.singlecell()
genes = data['Y']
labels = data['labels']
label_dict = dict(((i,l) for i,l in enumerate(labels)))

YFull = genes[strgene].values

N = genes.shape[0]
G = genes.shape[1]
genes.describe()
print genes.shape
stageCell = np.zeros(N)
stageN = np.zeros(N)
for i,l in enumerate(labels):
    stageCell[i] = int(l[:2])
    stageN[i] = np.log2(stageCell[i]) + 1
    
# Load pseudotime as estimated by Bayesian GPLVM (Max's method)
if(fPseudoTime):
    ptFull,YGPLVM = mouseQPCRModelSelection.LoadMouseQPCRData(subsetSelection=0)
    assert ptFull.size == stageCell.size, 'Pseudotime should be same size.  stageCell=' + str(stageCell.shape) + ' ptFull=' + str(ptFull.shape)
    assert YGPLVM.shape[0] == YFull.shape[0], 'Y shapes dont match YGPLVM=' + str(YGPLVM.shape) + ' YFull=' + str(YFull.shape)
    print 'Using pseudotime'
else:
    print 'Using capture times'
    ptFull = stageCell
    
print 'Doing map inference. Date shapes='
pt = ptFull[::subsetSelection].copy()
Y = YFull[::subsetSelection,None].copy()    
print pt.shape
print Y.shape

Doing subsets selection 3, looking at gene Id2 and pseudotime=1
Loading QPCR data
(437, 48)
Loaded data data/guo_ssData.p with nrowsXncols = (437, 2).
(437, 2)
(437,)
LoadMouseQPCRData output
(437, 2)
(437,)
Using pseudotime
Doing map inference. Date shapes=
(146,)
(146, 1)


In [2]:
t0 = time.time()
m,mV = mouseQPCRModelSelection.InitModels(pt,Y,nsparse=100)
print 'Times=%g secs'%(time.time()-t0)   

Branching kernel =====================
unnamed.branchkernelparam.kern.variance transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.kern.lengthscales transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.Bv transform:(none) prior:None
[[ 1.]]
unnamed.white.variance transform:+ve prior:None
[ 1.]
Created 100 inducing points in [0.0,89.0]
Initialise models: MAP =====================


ValueError: Dimensions 300 and 438 are not compatible

In [3]:
import branch_kernParamGPflow as bk
import BranchingTree as bt
import GPflow
import AssignGPGibbsSingleLoop
BvaluesInit = np.ones((1,1)) # initial values
tree = bt.BinaryBranchingTree(0,90,fDebug=False) # set to true to print debug messages
tree.add(None,1,10) # single branching point
(fm, _) = tree.GetFunctionBranchTensor()
Kbranch = bk.BranchKernelParam(GPflow.kernels.Matern32(1), fm, BvInitial=BvaluesInit) + GPflow.kernels.White(1)
# NB: fix the branching point if optimizing. Kbranch.branchkernelparam.Bv.fixed = True
print 'Branching kernel ====================='
print Kbranch

nsparse =10
if(nsparse > 0):
    l = np.min(pt)
    u = np.max(pt)
    Z = np.linspace(l,u,nsparse)
    print 'Created %g inducing points in [%.1f,%.1f]'%(nsparse,l,u)

print 'Initialise models: MAP ====================='
m = AssignGPGibbsSingleLoop.AssignGPGibbsFast(pt, Y, Kbranch,Z=Z)
m.CompileAssignmentProbability()

Branching kernel =====================
unnamed.branchkernelparam.kern.variance transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.kern.lengthscales transform:+ve prior:None
[ 1.]
unnamed.branchkernelparam.Bv transform:(none) prior:None
[[ 1.]]
unnamed.white.variance transform:+ve prior:None
[ 1.]
Created 10 inducing points in [0.0,89.0]
Initialise models: MAP =====================


ValueError: Dimensions 30 and 438 are not compatible

In [4]:
import tensorflow as tf
M = m.XExpanded.shape[0]
assignments_tf = tf.placeholder(tf.int32)

tau = 1./m.likelihood.variance
L = m.KChol
W = tf.matrix_triangular_solve(L, m.Kuf)
p = tf.Variable(tf.zeros((m.XExpanded.shape[0],), tf.float64))
p.assign(np.zeros((m.XExpanded.shape[0],)))  # make sure we overwrite previous run
p = tf.scatter_add(p, assignments_tf, np.ones(m.Y.shape[0]), name='scatteradd_p')

LTA = W * tf.sqrt(p)
P = tf.matmul(LTA, tf.transpose(LTA)) * tau + GPflow.tf_hacks.eye(M)

traceTerm = -0.5 * tau * (tf.reduce_sum(m.Kdiag*p) - tf.reduce_sum(tf.square(LTA)))

R = tf.cholesky(P)


In [19]:
M

438

In [5]:
PhiY = tf.Variable(tf.zeros((m.XExpanded.shape[0], m.Y.shape[1]), tf.float64))
PhiY = tf.scatter_add(PhiY, assignments_tf, m.Y)

In [9]:
KufPhiY = tf.matmul(m.Kuf,PhiY)
KufPhiY

<tf.Tensor 'MatMul_2:0' shape=(30, 1) dtype=float64>

In [11]:
R

<tf.Tensor 'Cholesky:0' shape=(30, 30) dtype=float64>

In [14]:
R = tf.cholesky(P)

PhiY = tf.Variable(tf.zeros((m.XExpanded.shape[0], m.Y.shape[1]), tf.float64))
PhiY = tf.scatter_add(PhiY, assignments_tf, m.Y) # N X 1
KufPhiY = tf.matmul(m.Kuf, PhiY) # I = ind pt X 1

LPhiY = tf.matmul(tf.transpose(L), KufPhiY)
RiLPhiY = tf.matrix_triangular_solve(R, LPhiY, lower=True)

a= traceTerm + 0.5*m.Y.size*tf.log(tau)\
    - 0.5*m.Y.shape[1]*tf.reduce_sum(tf.log(tf.square(tf.diag_part(R))))\
    - 0.5*tau*tf.reduce_sum(tf.square(m.Y))\
    + 0.5*tf.reduce_sum(tf.square(tau * RiLPhiY))
a

<tf.Tensor 'add_26:0' shape=(1,) dtype=float64>

In [ ]:
# t0 = time.time()
# logVBBound, logLike = mouseQPCRModelSelection.DoModelSelectionRuns(m,mV,Bpossible=np.array([20]), strSaveState='rawData'+str(fPseudoTime), \
#     fSoftVBAssignment=True, fOptimizeHyperparameters = False, fReestimateMAPZ=True,\
#     numMAPsteps = 10, fPlotFigure=True)
# print 'Times=%g secs'%(time.time()-t0)   